In [10]:
import pandas as pd
import os
import datetime
import json
from google.cloud import bigquery
from google.oauth2 import service_account
import numpy as np

In [11]:
def get_latest_file2(directory):
    files = [f for f in os.listdir(directory) 
             if f.endswith('.XLSX') and not f.startswith('~$') and f.startswith('Query')]
    if not files:
        raise FileNotFoundError(f"No Excel files found in {directory}")
    files = [os.path.join(directory, f) for f in files]
    latest_file = max(files, key=os.path.getmtime)
    print(f'Lendo {latest_file}')
    return latest_file

def inverter_data(data):
    if pd.isna(data):
        return np.nan
    
    if isinstance(data, str):
        if '/' in data:
            dia, mes, ano = data.split('/')
            return ano + mes + dia
        elif len(data) >= 10 and data[4] == '-' and data[7] == '-':
            return data[:10].replace('-', '')  # Converte 'yyyy-mm-dd' para 'yyyymmdd'
    
    return np.nan

In [12]:
caminho_credenciais = r'C:\Users\rogerio.mello\Documents\Repositorio\credenciais.json'

# Carregando arquivo JSON com credenciais
with open(caminho_credenciais, 'r') as f:
    credenciais_json = json.load(f)

credentials = service_account.Credentials.from_service_account_info(credenciais_json)
client = bigquery.Client(credentials=credentials, project=credenciais_json["project_id"])
print('Conexão ao GCP realizada com sucesso!')

Conexão ao GCP realizada com sucesso!


In [13]:
caminho_siglas = r'\\grupofleury\Auditoria\Auditoria_Interna\Auditoria_Continua\Rogerio.Mello\TRABALHOS REALIZADOS\2 - [ADITIVOS] Leitura de PDFs e Imagens, Comparativos, Criação de Interface\2.10 - OPAs sigla contratada'
data_atual = datetime.datetime.now()
data_formatada = data_atual.strftime("%d-%m-%Y")
data_hoje = pd.to_datetime(datetime.datetime.now().date()).strftime('%d/%m/%Y')
data_hoje = int(inverter_data(data_hoje))


In [14]:
siglas_df = pd.read_excel(caminho_siglas + r'\Siglas.xlsx', sheet_name='Sheet1')

In [15]:
siglas_df

,Siglas
0,RMANGIOABDOM
1,RMARTEABDOM
2,RMVENABDOM
3,RMARTMESENT
4,RMARTRENAIS
...,...
56,TCVENMS
57,TCARTMMSS
58,TCARTMMII
59,TCARTEMI


In [32]:
# Converter siglas do DataFrame para uma string formatada para SQL
siglas = "', '".join(siglas_df['Siglas'].astype(str).unique())
#SELECT DISTINCT TABELA_PRECO, SIGLA_PRODUTO, PRECO_CONVENIO, PRECO_PARTICULAR, QTD_CH, QTD_M2, VALOR_M2, VALOR_TOTAL_M2, VALOR_TOTAL_EXAME, PORTE
# Criar a query SQL
query = f"""
SELECT DISTINCT TABELA_PRECO, SIGLA_PRODUTO, PRECO_CONVENIO, PRECO_PARTICULAR, QTD_CH, QTD_M2, VALOR_M2, VALOR_TOTAL_M2, VALOR_TOTAL_EXAME, PORTE
FROM `data-lake-prd-276215.rz.BI_RZTB_TABELAS_PRECO_FINAL_PRICING` 
WHERE SIGLA_PRODUTO IN ('{siglas}');
"""

print("query")
requisicao = client.query(query)
print("to dataframe")
df5 = requisicao.result().to_dataframe()

query
to dataframe


c:\Users\rogerio.mello\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [33]:
df5

,TABELA_PRECO,SIGLA_PRODUTO,PRECO_CONVENIO,PRECO_PARTICULAR,QTD_CH,QTD_M2,VALOR_M2,VALOR_TOTAL_M2,VALOR_TOTAL_EXAME,PORTE
0,GRUPWASSIST-F,TCARTMMSS,2806.62,3226.0,2242.80,0.0,0.00,0.000,2242.800,0.00
1,SALUDVALORDF-V5,TCARTMMSS,2806.62,3226.0,2604.17,0.0,0.00,0.000,2604.170,0.00
2,REMATARDE-DM-V2,RMARTEMS,1844.40,2120.0,450.00,0.0,0.00,0.000,450.000,0.00
3,INTEPARTN-V1,TCARTMMSS,2806.62,3226.0,1752.90,0.0,0.00,0.000,1752.900,NaN
4,ATESTE2014-V3,TCARTMMSS,2806.62,3226.0,2080.00,0.0,0.00,0.000,2080.000,0.00
...,...,...,...,...,...,...,...,...,...,...
58017,PORTOBRON-FMV9,TCVENTORAX,997.02,1146.0,932.72,0.0,26.96,0.000,932.720,0.00
58018,GONCOCLIN-DI-1,TCVENTORAX,997.02,1146.0,484.00,0.0,0.00,0.000,484.000,0.00
58019,RGRANBAS-LAB-V6,TCVENTORAX,997.02,1146.0,538.86,1.5,31.59,47.385,586.245,160.43
58020,EUROCENTERF19,TCVENTORAX,997.02,1146.0,1331.80,0.0,31.59,0.000,1331.800,0.00


In [34]:
Query_sap_real_dir = fr'\\grupofleury\Dir_Exec_Suporte_Operacoes\GCC - Query SAP\2025\03 Março _2025'
Query_sap_real = get_latest_file2(Query_sap_real_dir)
df_sap = pd.read_excel(Query_sap_real, usecols=['Nome da sigla de contrato', 'Descrição Convênio', 'Descrição Plano', 'Tabela de Preços', 'Validade do plano até', 'Válido até', 'Descrição Empresa', 'Capitation'], keep_default_na=False, na_values=[])


###### TRATAMENTO QUERY_SAP
df_sap[['Validade do plano até', 'Válido até']] = (df_sap[['Validade do plano até', 'Válido até']].fillna('').astype(str))
df_sap['Validade do plano até'] = df_sap['Validade do plano até'].apply(inverter_data)
df_sap['Válido até'] = df_sap['Válido até'].apply(inverter_data)
df_sap[['Validade do plano até', 'Válido até']] = df_sap[['Validade do plano até', 'Válido até']].fillna(0).astype(int)
df_sap= df_sap[df_sap['Validade do plano até'] > data_hoje]
df_sap= df_sap[df_sap['Válido até'] > data_hoje]
df_sap = df_sap[df_sap['Capitation'] != 'S']


Lendo \\grupofleury\Dir_Exec_Suporte_Operacoes\GCC - Query SAP\2025\03 Março _2025\QuerySap 11_03_2025.XLSX


In [35]:
ativos = pd.merge(df5, df_sap, left_on='TABELA_PRECO', right_on='Tabela de Preços', how='left')

In [36]:
ativos.to_excel(caminho_siglas + fr'\Ativos_{data_formatada}.xlsx', index=False)

In [37]:
grupos = ativos.groupby("SIGLA_PRODUTO")
dfs = []
df_temp = pd.DataFrame()

limite_excel = 1048576

for _, grupo in grupos:
    if len(df_temp) + len(grupo) > limite_excel:
        dfs.append(df_temp)  # Salva o DataFrame atual
        df_temp = pd.DataFrame()  # Reinicia para um novo arquivo
    df_temp = pd.concat([df_temp, grupo])

# Adiciona o último DataFrame se não estiver vazio
if not df_temp.empty:
    dfs.append(df_temp)

In [38]:

# Salvar os arquivos
for i, df_parte in enumerate(dfs):
    df_parte.to_excel(caminho_siglas + fr'\Ativos_{data_formatada} {i+1}.xlsx', index=False, sheet_name="Dados")

In [23]:
ativos

,TABELA_PRECO,SIGLA_PRODUTO,Descrição Convênio,Descrição Empresa,Descrição Plano,Nome da sigla de contrato,Tabela de Preços,Validade do plano até,Válido até,Capitation
0,OMINCORPTAB2-F2,TCARTMMSS,Omint Assistencial Serviços de Saúde,Omint - Sem Valid,CORPORATE - Medicina Completo,OMINTSVAL2-F,OMINCORPTAB2-F2,99991231.0,99991231.0,
1,OMINCORPTAB2-F2,TCARTMMSS,Omint Assistencial Serviços de Saúde,Omint Assistencial Serviços de Saúde,CORPORATE - Medicina Completo,OMINTCORPOR-F,OMINCORPTAB2-F2,99991231.0,99991231.0,
2,SENADOSIS-F,TCARTMMSS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KIPPSAUDE-F5,TCARTMMSS,Omint Assistencial Serviços de Saúde,Atendimento Excepcional - KIPP,Atendimento Excepcional,OMINTKIPP2AE-F,KIPPSAUDE-F5,99991231.0,99991231.0,N
4,KIPPSAUDE-F5,TCARTMMSS,Omint Assistencial Serviços de Saúde,Kipp Saúde,Conforto mais,OMINTKIPP2-F,KIPPSAUDE-F5,99991231.0,99991231.0,
...,...,...,...,...,...,...,...,...,...,...
484713,SALVALOR-LAB-Fv,TCVENTORAX,Labs amais - Salud Valor,Salud Valor,Faturado 100% Seguradora,SALVALOR-LAB-F,SALVALOR-LAB-Fv,99991231.0,99991231.0,
484714,GONCOCLIN-LAB-V,TCVENTORAX,Labs amais - Grupo Oncoclínicas,Pesquisa Clinica,Pacientes,GONCOCLIN-LAB-F,GONCOCLIN-LAB-V,99991231.0,99991231.0,
484715,CLUDE-FM-V,TCVENTORAX,Felippe Mattoso - Clude,Clude,A Vista,CLUDE-FM-V,CLUDE-FM-V,99991231.0,99991231.0,
484716,DOCTMEIER-LABv1,TCVENTORAX,Labs amais - Doctor Med,Doctor Med Meier,Todos,DOCTMEIER-LAB-F,DOCTMEIER-LABv1,99991231.0,99991231.0,
